# URDF experiments for design on demand

URDF units are written and joined with the help of the ODIO URDF library and constructed robots are played around with using the teleop from pydrake

In [1]:
# python libraries
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

# pydrake imports
from pydrake.all import (MultibodyPlant, AddMultibodyPlantSceneGraph,
                         DiagramBuilder, Parser, VectorSystem, SignalLogger,
                         Simulator, PlanarSceneGraphVisualizer, Multiplexer,
                         plot_system_graphviz, MatrixGain, InverseKinematics, )

import pydrake.solvers.mathematicalprogram as mp

In [2]:
import meshcat
import pydrake.systems.meshcat_visualizer as meshcat_visualizer 

In [3]:
import os
from IPython.display import display
from ipywidgets import Textarea

In [4]:
from pydrake.all import RigidTransform, RotationMatrix, RollPitchYaw

import pydrake.multibody.jupyter_widgets
import pydrake.systems.jupyter_widgets

from manipulation.jupyter_widgets import MakePoseSlidersThatPublishOnCallback

In [333]:
from odio_urdf import *
import math
import sys
# from operator import add
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint

In [394]:
# Initiate the tree structure with a base

# Decided to use numpy for array manipulation

# Need to setup good connection rules for both origin and orientation
# And then setup joints

# We need a new class to describe the links and the joints
# The end product has to be disctributed as modules and submodules
# https://stackoverflow.com/questions/18555193/python-modules-with-submodules-and-functions

# For now we have the module class within the main class
class robotDoD:
    # A class to define all the modules and their description
    class robotModule:
        def __init__(self, name):
            self.name = name;
            # Define interfaces poition and orientation
            self.intA = np.array([[0,0,0],[0,0,0]]) # Interface A origin and orientation
            self.intB = np.array([[0,0,0],[0,0,0]]) # Interface B origin and orientation
            self.mesh_path = "" # Set the location of the path
            self.scale = "0.001 0.001 0.001" # Set scale as string
            self.type = 0 # 0-link; 1-joint    
#             self.default_orientation = np.array([0, 0, 0]) # If necessary to re-orient from the obtained meshes
#             self.default_origin = np.array([0,0,0]) # To re-locate from the default location of the mesh
  
    # A helper class for transformation
    class helperFuncs:
        def __init__(self):
            pass
            
        def list2space(self, com):
            return str(com[0])+' '+str(com[1])+' '+str(com[2])
    
        def rotZ(self, angle):
            return np.array([[np.cos(angle), -np.sin(angle), 0],[np.sin(angle), np.cos(angle), 0],[0,0,1]])

        def rotY(self, angle):
            return np.array([[np.cos(angle), 0, np.sin(angle)],[0, 1, 0],[-np.sin(angle), 0, np.cos(angle)]])

        def rotX(self, angle):
            return np.array([[1,0,0], [0, np.cos(angle), -np.sin(angle)],[0, np.sin(angle), np.cos(angle)]])
    
        def transform_vector(self, axis, angle, vector):
            if axis == 0:
                return self.rotX(angle).dot(vector)
            if axis == 1:
                return self.rotY(angle).dot(vector)
            if axis == 2:
                return self.rotZ(angle).dot(vector)
            else:
                raise AssertionError("Error: Rotation about unknown axis requested") 
                
        # Resolve the axis and angle when aligned
        def resolve_axis(self, module):
            def foo(x):
                return x[0]*x[1]*x[2]

            def non_lin(x):
                return np.linalg.norm(x)

            lin_constraint = LinearConstraint(module.intA[1], 0, 0)
            nonlin_constraint = NonlinearConstraint(non_lin, 1, 1)

            sol = minimize(foo,[1/np.sqrt(3),1/np.sqrt(3),1/np.sqrt(3)], constraints = [lin_constraint, nonlin_constraint])
            return sol['x']            
    
    def __init__(self, name):
        self.name = name
        # Initiate the helper class
        self.helper_funcs = self.helperFuncs()
        # Declare materials
        self.materials = Group(
            Material("blue", Color(rgba="0 0 0.8 1.0")),
            Material("red", Color(rgba="0.8 0 0 1.0")))  
        self.material_color = ["blue", "red"]
        self.module_types = ["linkMod", "jointMod"]
        # Declare the robot
        self.robot = Robot(self.name, self.materials)
        # Declare robots tree structure
        self.tree = {}
        self.prev_module = []
        
    def add_root(self):        
        # Add root module to the robot URDF
        root_mod = self.robotModule('root_mod')
        # Define the module
        root_mod.intA = np.array([[0,0,0],[0,0,-1]])
        root_mod.intB = np.array([[0,0,0.01],[0,0,1]])
        root_mod.type = 1
        
        root_link_urdf = Link(
                    Visual(
                        Origin(rpy = "0 0 0", xyz = self.helper_funcs.list2space(root_mod.intA[0])),
                        Material (name = self.material_color[root_mod.type]),
                        Geometry(Cylinder(length = 0.02, radius = 0.05))),
                    name = root_mod.name)
        
        self.prev_module = root_mod.intB
        self.robot(root_link_urdf)
        self.tree[root_mod.name] = root_mod.intB
    
    def add_module(self, module):
        # Always start from the root module which initates the self.prev_module
        # Check if prev_module is empty => root not initiated 
        if not self.prev_module.any():
            raise AssertionError("Error: Add root module first before initiating the tree")
        # Get info from self.prev_module
        # Compute connection rules from prev and current modules
        current_origin, current_orientation = self.computeConnection(module)
        
        # Generate module URDF
        module_urdf = Link(
                         Visual(
                            Origin(xyz = self.helper_funcs.list2space(current_origin), rpy = self.helper_funcs.list2space(current_orientation)),
                            Material(name = self.material_color[module.type]),
                            Geometry(Mesh(filename = module.mesh_path, scale = module.scale))),
                        name= self.module_types[module.type]+ '_'+str(len(self.tree.keys())))
        
        # Add the module to the robot URDF
        self.robot(module_urdf)
        # Save all the origins and orientations in a tree
        self.tree[module.name] = self.prev_module
        pass

    def constructURDF(self):
        # Later replace the name of the URDF to the given name
        name = 'test.urdf'
        with open(name, 'w') as f:
            print(self.robot, file=f)
            print('Robot design successfully saved as '+name)
    
    def visualiseRobot(self, urdf_path):
        # Loading the updated robot URDF
        builder = DiagramBuilder()
        robot, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
        Parser(robot).AddModelFromFile(urdf_path)
        robot.Finalize()
        
        diagram = builder.Build()

        diagram_context = diagram.CreateDefaultContext()
        plant_context = diagram.GetSubsystemContext(robot, diagram_context)

        meshcat.Visualizer().delete()
        # delete_prefix_on_load=True removes any previous meshes when a new one is loaded
        visualizer = meshcat_visualizer.ConnectMeshcatVisualizer(
            builder, 
            scene_graph, 
            delete_prefix_on_load=False)
        
        visualizer.load()
        
    def computeConnection(self, module):
        current_orientation = np.zeros(3)
        # Compute the rotation axis
        temp = np.cross(self.prev_module[1], module.intA[1])
        sinangle = np.linalg.norm(temp)
        cosangle = np.round(np.dot(self.prev_module[1], module.intA[1]))
        # Hangle sinangle being zero
        print(module.name)
        if sinangle == 0 and cosangle == 1:
            # We need the vector flipped
            # We need to resolve redundancy here by solving an optimisation problem
            rotaxis = self.helper_funcs.resolve_axis(module)
            rotangle = np.pi
            
        if sinangle == 0 and cosangle == -1:
            rotangle = 0
            rotaxis = np.zeros(3)
            
        if sinangle != 0:
            rotaxis = temp/sinangle
            # Compute the rotation angle
            rotangle = np.arctan2(sinangle, cosangle)
        
        # print(module.name,' ', rotaxis, ' ', rotangle)
        # Compute the quaternion
        r = R.from_rotvec(rotangle*rotaxis)
        # Convert to Euler rotation 'xyz'
        current_orientation = r.as_euler('xyz')

        # Rotation matrix
        rmat = r.as_matrix()
        current_origin = self.prev_module[0] + np.dot(rmat,(-module.intA[0]))
        # Next orientation
        next_orientation = np.dot(rmat, module.intB[1])
        next_origin = current_origin + np.dot(rmat, module.intB[0])
        # Update prev_module        
        self.prev_module = np.array([next_origin, next_orientation])
    
        return current_origin, current_orientation

In [444]:
# Declare the robotDoD class
roboDoD = robotDoD("test");

In [445]:
# Declare all the modules
# Keep this database of modules independant of the experiment being done

# jmod
jmod = robotDoD.robotModule('jmod')
# Define the module
jmod.intA = np.array([[0,0,0],[0,-1,0]])
jmod.intB = np.array([[0.031, 0.0556, 0],[1,0,0]])
jmod.type = 1
jmod.mesh_path = "./meshes/j_module.obj"

# l1mod
l1mod = robotDoD.robotModule('l1mod')
# Define the module
l1mod.intA = np.array([[0.04,0,0],[1,0,0]])
l1mod.intB = np.array([[0.04,0.15,0],[1,0,0]])
l1mod.type = 0
l1mod.mesh_path = "./meshes/l1_module.obj"

# l2mod
l2mod = robotDoD.robotModule('l2mod')
# Define the module
l2mod.intA = np.array([[0.04,0,0],[1,0,0]])
l2mod.intB = np.array([[0,0.15,0],[0,1,0]])
l2mod.type = 0
l2mod.mesh_path = "./meshes/l2_module.obj"

# l3mod
l3mod = robotDoD.robotModule('l3mod')
# Define the module
l3mod.intA = np.array([[0,0.15,0],[0,-1,0]])
l3mod.intB = np.array([[0.04, 0, 0],[1,0,0]])
l3mod.type = 0
l3mod.mesh_path = "./meshes/l3_module.obj"

In [458]:
import time
components = [jmod, l1mod, jmod, l2mod, jmod, l3mod, jmod, l2mod]
for i in range(9):
    
    roboDoD = robotDoD("test");
    
    jmod = robotDoD.robotModule('jmod')
    # Define the module
    jmod.intA = np.array([[0,0,0],[0,-1,0]])
    jmod.intB = np.array([[0.031, 0.0556, 0],[1,0,0]])
    jmod.type = 1
    jmod.mesh_path = "./meshes/j_module.obj"

    # l1mod
    l1mod = robotDoD.robotModule('l1mod')
    # Define the module
    l1mod.intA = np.array([[0.04,0,0],[1,0,0]])
    l1mod.intB = np.array([[0.04,0.15,0],[1,0,0]])
    l1mod.type = 0
    l1mod.mesh_path = "./meshes/l1_module.obj"

    # l2mod
    l2mod = robotDoD.robotModule('l2mod')
    # Define the module
    l2mod.intA = np.array([[0.04,0,0],[1,0,0]])
    l2mod.intB = np.array([[0,0.15,0],[0,1,0]])
    l2mod.type = 0
    l2mod.mesh_path = "./meshes/l2_module.obj"

    # l3mod
    l3mod = robotDoD.robotModule('l3mod')
    # Define the module
    l3mod.intA = np.array([[0,0.15,0],[0,-1,0]])
    l3mod.intB = np.array([[0.04, 0, 0],[1,0,0]])
    l3mod.type = 0
    l3mod.mesh_path = "./meshes/l3_module.obj"
    
    roboDoD.add_root()
    
    for j in range(i):
        roboDoD.add_module(components[j])
    
    roboDoD.constructURDF()
    roboDoD.visualiseRobot("./test.urdf")
    time.sleep(0.5)

Robot design successfully saved as test.urdf
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7077/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
jmod
jmod   [ 1.  0. -0.]   1.5707963267948966
Robot design successfully saved as test.urdf
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7078/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
jmod
jmod   [ 1.  0. -0.]   1.5707963267948966
l1mod
l1mod   [0.         0.70710678 0.70710679]   3.141592653589793
Robot design successfully saved as test.urdf
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7079/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can

In [446]:
# Now we have a base robotDoD class and now adding modules one by one
roboDoD.add_root()
roboDoD.add_module(jmod)
# roboDoD.add_module(l2mod)
# roboDoD.add_module(jmod)
roboDoD.add_module(l1mod)
roboDoD.add_module(jmod)
roboDoD.add_module(l2mod)
roboDoD.add_module(jmod)
roboDoD.add_module(l3mod)
roboDoD.add_module(jmod)
roboDoD.add_module(l2mod)

jmod
jmod   [ 1.  0. -0.]   1.5707963267948966
l1mod
l1mod   [0.         0.70710678 0.70710679]   3.141592653589793
jmod
jmod   [1.24977041e-08 0.00000000e+00 1.00000000e+00]   1.5707963267948966
l2mod
l2mod   [ 0.00000000e+00  1.24977041e-08 -1.00000000e+00]   1.5707963267948966
jmod
jmod   [-1.24977041e-08 -0.00000000e+00 -1.00000000e+00]   1.5707963267948966
l3mod
l3mod   [ 1.00000000e+00  0.00000000e+00 -4.44170791e-09]   1.2497704121950138e-08
jmod
jmod   [-3.46880972e-25 -0.00000000e+00 -1.00000000e+00]   1.5707963267948966
l2mod
l2mod   [-0.00000000e+00  3.46880972e-25  1.00000000e+00]   1.5707963267948966


In [447]:
# Now construct the URDF
roboDoD.constructURDF()

Robot design successfully saved as test.urdf


In [448]:
# Print the URDF for eef_link name
roboDoD.robot();

In [449]:
# Loading the updated robot URDF
roboDoD.visualiseRobot("./test.urdf")

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7041/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [413]:
jmod.intB[0]

array([0.031 , 0.0556, 0.    ])

In [391]:
roboDoD.tree

{'root_mod': array([[0.  , 0.  , 0.01],
        [0.  , 0.  , 1.  ]]),
 'jmod': array([[ 3.100000e-02,  1.234568e-17, -4.560000e-02],
        [ 1.000000e+00,  0.000000e+00,  0.000000e+00]]),
 'l1mod': array([[-4.90000019e-02, -4.06399673e-10,  1.04400000e-01],
        [-1.00000000e+00, -1.24977042e-08,  1.24977041e-08]])}

In [375]:
roboDoD.prev_module

array([[ 3.09999981e-02,  5.93416659e-10,  1.04399999e-01],
       [-1.00000000e+00, -1.24977042e-08,  1.24977041e-08]])

In [376]:
# Find the axis and angle between the modules
# root_mod and jmod, since all the int orientations are unit vectors cross product has to be unit?
temp = np.cross(roboDoD.prev_module[1], l1mod.intA[1])
sinangle = np.linalg.norm(temp)
# rotaxis = temp/sinangle
print(sinangle)

cosangle = np.dot(roboDoD.prev_module[1], l1mod.intA[1])
rotangle = np.arctan2(sinangle, cosangle)
print(cosangle)
# # Now define the quaternion
# r = R.from_quat(np.append(np.cos(rotangle/2), np.sin(rotangle/2)*rotaxis))
# r.as_euler('xyz')

1.767442269173094e-08
-0.9999999999999996


In [377]:
tempq = np.append(np.cos(rotangle/2), rotaxis*np.sin(rotangle/2))
print(tempq)

[8.83721125e-09            nan            nan            nan]


In [378]:
r = R.from_quat(tempq)
r.as_euler('xyz')

ValueError: Found zero norm quaternions in `quat`.

In [348]:
r = R.from_rotvec(0*np.array([0,0,0]))

In [349]:
r.as_euler('xyz')

array([0., 0., 0.])

In [334]:
def foo(x):
    return x[0]*x[1]*x[2]

def non_lin(x):
    return np.linalg.norm(x)

from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
lin_constraint = LinearConstraint(l1mod.intA[1], 0, 0)
nonlin_constraint = NonlinearConstraint(non_lin, 1, 1)

sol = minimize(foo,[1/np.sqrt(3),1/np.sqrt(3),1/np.sqrt(3)], constraints = [lin_constraint, nonlin_constraint])

In [337]:
sol['x']

array([0.        , 0.70710678, 0.70710679])

In [ ]:
# Fix the l1 offset issue
# Manipulate robot FK
# Manipulate robot IK
# Add a taskspace marker